In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
import joblib
import os

In [3]:
dx=pd.read_csv('/content/real_legitimate_v1.csv')

In [4]:
dy=pd.read_csv('/content/real_malware_v1.csv')

In [5]:
df = pd.concat([dx, dy], ignore_index=True)
df.to_csv('combined.csv', index=False)

In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df=df.to_csv('shuffled_dataset.csv', index=False)

In [7]:
dr=pd.read_csv('/content/shuffled_dataset.csv')

In [8]:
data={
    'Package':dr['Package'][:5000],
    'Malware':dr['Malware'][:5000],
    'prctl':dr['prctl'][:5000],
    'sha256':dr['sha256'][:5000],
    'EarliestModDate':dr['EarliestModDate'][:5000],
    'Malware':dr['Malware'][:5000],
    'write': dr['write'][:5000],
    'read':dr['read'][:5000],
    'write':dr['write'][:5000],
    'munmap':dr['munmap'][:5000],
    'mprotect':dr['mprotect'][:5000],
    'madvise':dr['madvise'][:5000],
    'ioctl':dr['ioctl'][:5000],
    'writev':dr['writev'][:5000],
    'dup':dr['dup'][:5000],
    'lseek':dr['lseek'][:5000],
    'mmap2':dr['mmap2'][:5000],
    'fstat64':dr['fstat64'][:5000],
    'rt_sigprocmask':dr['rt_sigprocmask'][:5000],
   'recvfrom':dr['recvfrom'][:5000],
    'getsockopt':dr['getsockopt'][:5000],
    'lseek':dr['lseek'][:5000],
    'clock_gettime':dr['clock_gettime'][:5000],
    'futex':dr['futex'][:5000],
    'pread':dr['pread'][:5000],
     'FilesInsideAPK':dr['FilesInsideAPK'][:5000],
   'TimesSubmitted':dr['TimesSubmitted'][:5000],
    'getsockopt':dr['getsockopt'][:5000],
    'Activities':dr['Activities'][:5000],
    'clock_gettime':dr['clock_gettime'][:5000],
    'NrIntActivities':dr['NrIntActivities'][:5000],
    'NrIntActivitiesActions':dr['NrIntActivitiesActions'][:5000],
    'TotalIntentFilters':dr['TotalIntentFilters'][:5000],
    'Scanners':dr['Scanners'][:5000],
    'MalFamily':dr['MalFamily'][:5000]
    }
dg=pd.DataFrame(data)
dg

,Package,Malware,prctl,sha256,EarliestModDate,write,read,munmap,mprotect,madvise,...,futex,pread,FilesInsideAPK,TimesSubmitted,Activities,NrIntActivities,NrIntActivitiesActions,TotalIntentFilters,Scanners,MalFamily
0,com.kandy.ui\n,0,24,30eb265ef5a4a3bccd90a0f3dc9a12f03b9aba9eed59c1...,1/27/2011,7,7,91,177,23,...,31,99,112.0,4.0,12,1,1,1,66.0,NaN
1,com.chillaxing.easynews\n,0,51,2f4dfd247ff9d184bd48c4462d83fa8ca60219656c05c3...,6/5/2011,6,3,62,189,41,...,38,52,69.0,6.0,13,2,3,4,66.0,NaN
2,com.lnm.dzdaily\n,1,9,87490a66dadf3c497e75d1145fd54192c87a063a313825...,11/14/2013,191,91,1,6,6,...,74,2,27.0,15.0,4,1,1,2,65.0,Youmi
3,com.xyhelper\n,1,83,d31fc10c0db6a39a9d13e21476c6b866497835c6150019...,9/16/2016,6,12,55,589,42,...,90,72,10.0,3.0,2,2,2,2,64.0,SMSreg
4,cn.adouming.apals\n,1,39,d2672d4917e43e4951fbbddb92ce8b31896b96e463897a...,5/3/2012,10,9,40,183,32,...,47,5,27.0,25.0,4,1,1,2,65.0,Kuguo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,com.alibaba.aliexpresshd\n,0,136,be45048408e29060043a449e1439a6df98d389ea84f6e7...,1980-00-00,19,64,66,242,93,...,392,113,8032.0,14.0,221,78,156,106,56.0,NaN
4996,com.androiddoctor.battery\n,1,83,26fbec09b4196260dca8ef94d2605dba8d4db6bb869a17...,8/30/2011,7,12,36,590,41,...,125,38,98.0,125.0,14,1,1,4,65.0,FakeDoc
4997,com.alaskajim.numberones\n,0,30,da93311cb4f7f34156d7092f59c9a484e8870b85d4a8bd...,4/1/2011,3,9,6,18,14,...,23,12,69.0,2.0,5,1,1,1,65.0,NaN
4998,nl.vpro.drievoortwaalf.luisterpaal\n,0,18,7a4ef5bb3518a6bcc25cb8d2a2e2ba56526dacd7b17b1b...,3/28/2011,21,20,44,168,18,...,7,7,49.0,4.0,5,1,1,1,66.0,NaN


In [9]:
dg.isnull().sum()

Package                      0
Malware                      0
prctl                        0
sha256                       0
EarliestModDate              0
write                        0
read                         0
munmap                       0
mprotect                     0
madvise                      0
ioctl                        0
writev                       0
dup                          0
lseek                        0
mmap2                        0
fstat64                      0
rt_sigprocmask               0
recvfrom                     0
getsockopt                   0
clock_gettime                0
futex                        0
pread                        0
FilesInsideAPK               0
TimesSubmitted               0
Activities                   0
NrIntActivities              0
NrIntActivitiesActions       0
TotalIntentFilters           0
Scanners                     0
MalFamily                 2243
dtype: int64

In [10]:
dg=dg.fillna(0)

In [11]:
dg.isnull().sum()

Package                   0
Malware                   0
prctl                     0
sha256                    0
EarliestModDate           0
write                     0
read                      0
munmap                    0
mprotect                  0
madvise                   0
ioctl                     0
writev                    0
dup                       0
lseek                     0
mmap2                     0
fstat64                   0
rt_sigprocmask            0
recvfrom                  0
getsockopt                0
clock_gettime             0
futex                     0
pread                     0
FilesInsideAPK            0
TimesSubmitted            0
Activities                0
NrIntActivities           0
NrIntActivitiesActions    0
TotalIntentFilters        0
Scanners                  0
MalFamily                 0
dtype: int64

In [12]:
dg.dtypes

Package                    object
Malware                     int64
prctl                       int64
sha256                     object
EarliestModDate            object
write                       int64
read                        int64
munmap                      int64
mprotect                    int64
madvise                     int64
ioctl                       int64
writev                      int64
dup                         int64
lseek                       int64
mmap2                       int64
fstat64                     int64
rt_sigprocmask              int64
recvfrom                    int64
getsockopt                  int64
clock_gettime               int64
futex                       int64
pread                       int64
FilesInsideAPK            float64
TimesSubmitted            float64
Activities                 object
NrIntActivities            object
NrIntActivitiesActions     object
TotalIntentFilters         object
Scanners                  float64
MalFamily     

In [13]:
dg.dtypes

Package                    object
Malware                     int64
prctl                       int64
sha256                     object
EarliestModDate            object
write                       int64
read                        int64
munmap                      int64
mprotect                    int64
madvise                     int64
ioctl                       int64
writev                      int64
dup                         int64
lseek                       int64
mmap2                       int64
fstat64                     int64
rt_sigprocmask              int64
recvfrom                    int64
getsockopt                  int64
clock_gettime               int64
futex                       int64
pread                       int64
FilesInsideAPK            float64
TimesSubmitted            float64
Activities                 object
NrIntActivities            object
NrIntActivitiesActions     object
TotalIntentFilters         object
Scanners                  float64
MalFamily     

In [14]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_values = label_encoder.fit_transform(dg['MalFamily'].astype(str))
encoded_values = encoded_values.reshape(-1, 1)
dg['MalFamily'] = encoded_values
dg['MalFamily'].max()
encoded=label_encoder.fit_transform(dg['Package'].astype(str))
dg['Package']=encoded

In [15]:
dg.isnull().sum()

Package                   0
Malware                   0
prctl                     0
sha256                    0
EarliestModDate           0
write                     0
read                      0
munmap                    0
mprotect                  0
madvise                   0
ioctl                     0
writev                    0
dup                       0
lseek                     0
mmap2                     0
fstat64                   0
rt_sigprocmask            0
recvfrom                  0
getsockopt                0
clock_gettime             0
futex                     0
pread                     0
FilesInsideAPK            0
TimesSubmitted            0
Activities                0
NrIntActivities           0
NrIntActivitiesActions    0
TotalIntentFilters        0
Scanners                  0
MalFamily                 0
dtype: int64

In [16]:
dg=dg.fillna(0)

In [17]:
import pandas as pd

In [18]:
if pd.isnull(dg['EarliestModDate']).any():
    # Handle or fill the missing values if necessary
    dg['EarliestModDate'].fillna('fill_value', inplace=True)  # Replace 'fill_value' with an appropriate value
# Convert the 'EarliestModDate' column to datetime
dg['EarliestModDate'] = pd.to_datetime(dg['EarliestModDate'], errors='coerce')

In [19]:
print(dg['EarliestModDate'].dtype)

datetime64[ns]


In [20]:
dg.isnull().sum()

Package                    0
Malware                    0
prctl                      0
sha256                     0
EarliestModDate           89
write                      0
read                       0
munmap                     0
mprotect                   0
madvise                    0
ioctl                      0
writev                     0
dup                        0
lseek                      0
mmap2                      0
fstat64                    0
rt_sigprocmask             0
recvfrom                   0
getsockopt                 0
clock_gettime              0
futex                      0
pread                      0
FilesInsideAPK             0
TimesSubmitted             0
Activities                 0
NrIntActivities            0
NrIntActivitiesActions     0
TotalIntentFilters         0
Scanners                   0
MalFamily                  0
dtype: int64

In [21]:
dg=dg.fillna(0)

In [22]:
dg.isnull().sum()

Package                   0
Malware                   0
prctl                     0
sha256                    0
EarliestModDate           0
write                     0
read                      0
munmap                    0
mprotect                  0
madvise                   0
ioctl                     0
writev                    0
dup                       0
lseek                     0
mmap2                     0
fstat64                   0
rt_sigprocmask            0
recvfrom                  0
getsockopt                0
clock_gettime             0
futex                     0
pread                     0
FilesInsideAPK            0
TimesSubmitted            0
Activities                0
NrIntActivities           0
NrIntActivitiesActions    0
TotalIntentFilters        0
Scanners                  0
MalFamily                 0
dtype: int64

In [44]:
# %% [code]
#!/usr/bin/env python
# coding: utf-8
# # Experiment 3: Train in batches by month and test in the next month
import pandas as pd
import numpy as np
# initial year defines the training (< initial_year) and testing sets (>= initial_year)
initial_year = "2017-12-30"
# Transform submission_date in a datetime column:
dg['EarliestModDate'] =  pd.to_datetime(dg['EarliestModDate'],errors='coerce')
# In[7]:
# Group data by months and create subsets to train and test:
dataTrain =dg[dg['EarliestModDate'] <=initial_year]
dataTest = dg[dg['EarliestModDate'] >initial_year]
print("Train shape:", dataTrain.shape)
print("Test shape:", dataTest.shape)
# In[9]:

# In[11]:




# In[9]:


# Feature extraction method:
# Feature extraction method:
def feature_extraction(X_dataTrain, X_dataTest):
    from sklearn.feature_extraction.text import TfidfVectorizer
    # initialize X_train and X_test
    X_train = []
    X_test = []
    # save models used
    models = []
    # iterate over each column of X_dataTrain
    for i in range(X_dataTrain.shape[1]):
        # train feature word2vec using column i
        #print("Training column {}...".format(i))
        # get train and test data
        train_data = np.asarray(X_dataTrain[:, i], dtype=str)  # Convert to numpy array
        test_data = np.asarray(X_dataTest[:, i], dtype=str)  # Convert to numpy array
        # initialize and train model
        tfidf = TfidfVectorizer(max_features=100)
        tfidf.fit(train_data)
        # transform train and test texts to w2v mean
        train_tfidf = tfidf.transform(train_data).todense()
        test_tfidf = tfidf.transform(test_data).todense()
        # if first execution, save only features
        if len(X_train) == 0:
            X_train = train_tfidf
            X_test = test_tfidf
        # concatenate existing features
        else:
            X_train = np.concatenate((X_train, train_tfidf), axis=1)
            X_test = np.concatenate((X_test, test_tfidf), axis=1)
        # save model
        models.append(tfidf)
    return X_train, X_test

# In[11]:


# Normalization method:
def normalization(X_train, X_test):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(np.asarray(X_train))
    XX_train = scaler.transform(np.asarray(X_train))
    XX_test = scaler.transform(np.asarray(X_test))
    return XX_train, XX_test



# In[13]:


# Iterate over months in test data:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
#from sklearn.ensemble import RandomForestClassifier as Classifier
from sklearn.linear_model import SGDClassifier as Classifier
# group by year-month
dataTestTime = dataTest.groupby([dataTest.EarliestModDate.dt.year, dataTest.EarliestModDate.dt.month])
# get data train labels
y_train = np.array(dataTrain["Malware"])
# remove unused columns from dataTrain
UNUSED_COLUMNS = ["Malware", "sha256", "EarliestModDate"]
for c in UNUSED_COLUMNS:
    del dataTrain[c]
# get dataTrain values
X_dataTrain = dataTrain.values
# arrays to save information
accs = [] # accuracy
f1s = [] # f1score
recall = [] # recall
precision = [] # precision
years = []
months = []
# iterate over year and month
for (year, month), dataTestMonth in dataTestTime:
    # get data test labels
    y_test = np.array(dataTestMonth["Malware"])
    # remove unused columns from data test
    for c in UNUSED_COLUMNS:
        del dataTestMonth[c]
    # get dataTestMonth values
    X_dataTest = dataTestMonth.values
    # feature extraction
    X_train, X_test = feature_extraction(X_dataTrain, X_dataTest)
    # normalization
    XX_train, XX_test = normalization(X_train, X_test)
    # classifier
    # clf = RandomForestClassifier(random_state=0)
    clf = Classifier(random_state=0)
    # train classifier
    clf.fit(XX_train,y_train)
    # predict test data
    y_pred = clf.predict(XX_test)
    # update train data and labels with actual month
    X_dataTrain = np.append(X_dataTrain, X_dataTest, axis=0)
    y_train = np.concatenate((y_train, np.array(y_test)), axis=0)
    print("{}-{}".format(year,month))
    # save metrics
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    re = recall_score(y_test, y_pred)
    pr = precision_score(y_test, y_pred)
    if (f1 !=0) and (re != 0) and (pr != 0):
        accs.append(acc)
        f1s.append(f1)
        recall.append(re)
        precision.append(pr)
        years.append(year)
        months.append(month)
        print(acc)




Train shape: (4745, 30)
Test shape: (166, 30)
2017-12
1.0
2018-1
0.5294117647058824
2018-2
0.9545454545454546
2018-3
0.9047619047619048
2018-4
1.0
2018-5
0.88
2018-6
0.8888888888888888
2018-7
0.3333333333333333
2018-8


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2018-9


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2018-10
0.5
2018-11


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2019-3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2019-5
0.5
2019-10
1.0
2019-11
0.2
2020-4
0.6
2020-5
0.8333333333333334
2020-6
1.0
2020-11
1.0
2107-11


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
